# This project implemented by Elham Mahdipour
## She is a Ph.D. Candidate of computer engineering at Yazd University, Yazd, Iran.
### Please feel free and contact to me: elham.mahdipour@gmail.com/ elham.mahdipour@stu.yazd.ac.ir

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_leda('biogrid samples\SCerevisiae.gw')
G1

In [3]:
G2=nx.read_leda('biogrid samples\AThaliana.gw')
G2

In [4]:
### Check and Swap if G1 > G2 ###
print(len(G1))
print(len(G2))
print('*******')
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

5831
5897
*******
5831
5897


In [5]:
G_target_na=nx.read_weighted_edgelist('biogrid samples\SC-AT.tsv')
print(len(G_target_na))
G_target_na  

1440


In [6]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [7]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [8]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [9]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [10]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [11]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [12]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [13]:
elt=G_target_na.edges()
elt=list(elt)
print(len(elt))
elt

720


[('850996', '833987'),
 ('854309', '834370'),
 ('850997', '840114'),
 ('850998', '819918'),
 ('855386', '829969'),
 ('851342', '844228'),
 ('853757', '821995'),
 ('855723', '831311'),
 ('852439', '820937'),
 ('855104', '834747'),
 ('852383', '832836'),
 ('851954', '829838'),
 ('851065', '831077'),
 ('852456', '843098'),
 ('851098', '825755'),
 ('851102', '814698'),
 ('851582', '837733'),
 ('855474', '817673'),
 ('852665', '826665'),
 ('855428', '821628'),
 ('853371', '817259'),
 ('856402', '837734'),
 ('856413', '825385'),
 ('851117', '830044'),
 ('854986', '839636'),
 ('851125', '825833'),
 ('851631', '835714'),
 ('854025', '844399'),
 ('856815', '820218'),
 ('854070', '839916'),
 ('856434', '819849'),
 ('852231', '820293'),
 ('851139', '828392'),
 ('856462', '836220'),
 ('856466', '839521'),
 ('856467', '824778'),
 ('856473', '841952'),
 ('856474', '826596'),
 ('851114', '823849'),
 ('856844', '844406'),
 ('854930', '835835'),
 ('850505', '837023'),
 ('850369', '830541'),
 ('854844',

In [14]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=(nd1[i],nd2[j]) in elt
            if q1==False:
                c=0
            else:
               # print(q1)
                c=1
            seq[i][j]=c    
            
    return seq

In [15]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd, seq_sc

In [16]:
import numpy as np
m, coeff_pr, dd, cd, seq=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, sequence similarity, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(0)/TN or no(1)/TP]

In [17]:
en_mat=[]

for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
        else:
            align_class='No'
        
        sample=[int(nd1[i]),int(nd2[j]),seq[i][j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
        en_mat.append(sample)
print(len(en_mat))

34385407


In [18]:
yc=[]
noc=[]
for i in range(len(en_mat)):
    if en_mat[i][7]=='Yes': 
        yc.append(en_mat[i])
    else:
        noc.append(en_mat[i])
print(len(yc), len(noc))

265 34385142


In [19]:
from sklearn.utils import shuffle
data=yc+noc 
data = shuffle(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:7])
    y.append(data[i][7])
    
len(X),len(y)

(34385407, 34385407)

In [20]:
X_train=X
y_train=y

# test model for real data

In [21]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.


In [22]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)

# Phase 3: R-NA network

# test R-NA with tuned model

In [26]:
from keras.models import load_model
#del model
model_rnn = load_model('biogrid samples/deep_model_resample_RNN_SC-AT.h5')
model_rnn.load_weights('biogrid samples/deep_model_resample_RNN_weights_SC-AT.h5', by_name=True)

In [27]:
# without resample
result_tr = model_rnn.evaluate(X_train, y_train)
result_tr

34385407/34385407 [==============================] - 1922s 56us/step


[3.576287938195314e-06, 1.0, 3.5731545722228475e-06, 1.2718093071739656e-11]

In [28]:
%%time
out_tr=model_rnn.predict(X_train)
out_tr

# Create alignment file 
f = open("biogrid samples/align-SC-AT.txt", "w")
for i in range(len(out_tr)):
    if out_tr[i][1]>out_tr[i][0]:
        z=[X_train[i][0],X_train[i][1]]
        f.write(str(z))
f.close()

from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), out_tr.argmax(axis=1)))

from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), out_tr.argmax(axis=1) , average="macro"))

[[34385142        0]
 [       0      265]]
1.0
1.0
1.0
Wall time: 19min 30s


# test other classifier without resample

In [29]:
x_tra,y_tra=X_train,y_train

In [30]:
X_train,y_train=X,y

In [31]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


Accuracy of LDA classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999926713096634
[[34385142        0]
 [     252       13]]
0.9999963356534464
0.5245283018867924
0.546760757748067
Wall time: 6min


In [32]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of KNN classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999922932423048
[[34385142        0]
 [     265        0]]


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4999961466211524
0.5


C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.4999980733031519
Wall time: 41min 7s


In [33]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))

from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

Accuracy of SVM classifier on training set: 1.00
['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999862735613807
[[34385142        0]
 [     265        0]]
0.4999892454331463
0.5
0.4999970345531812
Wall time: 1h 34min 42s
